In [1]:
import torch
import sys
sys.path.insert(1, '../')

In [2]:
from models.resnet_simclr import ResNetSimCLR
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier
import yaml

In [3]:
folder_name = 'Mar10_21-50-05_thallessilva'
checkpoints_folder = os.path.join('../runs', folder_name, 'checkpoints')
config = yaml.load(open(os.path.join(checkpoints_folder, "config.yaml"), "r"), Loader=yaml.FullLoader)

In [4]:
config

{'batch_size': 512,
 'out_dim': 256,
 's': 1,
 'temperature': 0.5,
 'base_convnet': 'resnet18',
 'use_cosine_similarity': True,
 'epochs': 50,
 'num_workers': 4,
 'valid_size': 0.05,
 'eval_every_n_epochs': 2}

In [5]:
def _load_stl10(prefix="train"):
    X_train = np.fromfile('../data/stl10_binary/' + prefix + '_X.bin', dtype=np.uint8)
    y_train = np.fromfile('../data/stl10_binary/' + prefix + '_y.bin', dtype=np.uint8)

    X_train = np.reshape(X_train, (-1, 3, 96, 96))
    X_train = np.transpose(X_train, (0, 3, 2, 1))
    print("{} images".format(prefix))
    print(X_train.shape)
    print(y_train.shape)
    return X_train, y_train - 1

In [6]:
# load STL-10 train data
X_train, y_train = _load_stl10("train")

train images
(5000, 96, 96, 3)
(5000,)


In [7]:
# load STL-10 test data
X_test, y_test = _load_stl10("test")

test images
(8000, 96, 96, 3)
(8000,)


## Test protocol #1 PCA features

In [8]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

In [9]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train.reshape((X_train.shape[0],-1)))

pca = PCA(n_components=config['out_dim'])

X_train_pca = pca.fit_transform(scaler.transform(X_train.reshape(X_train.shape[0], -1)))
X_test_pca = pca.transform(scaler.transform(X_test.reshape(X_test.shape[0], -1)))

print("PCA features")
print(X_train_pca.shape)
print(X_test_pca.shape)

PCA features
(5000, 256)
(8000, 256)


In [10]:
def linear_model_eval(X_train, y_train, X_test, y_test):
    
    clf = LogisticRegression(random_state=0, max_iter=1000, solver='lbfgs', C=1.0)
    clf.fit(X_train, y_train)
    print("Logistic Regression feature eval")
    print("Train score:", clf.score(X_train, y_train))
    print("Test score:", clf.score(X_test, y_test))
    
    print("-------------------------------")
    neigh = KNeighborsClassifier(n_neighbors=10)
    neigh.fit(X_train, y_train)
    print("KNN feature eval")
    print("Train score:", neigh.score(X_train, y_train))
    print("Test score:", neigh.score(X_test, y_test))

In [11]:
linear_model_eval(X_train_pca, y_train, X_test_pca, y_test)

Logistic Regression feature eval
Train score: 0.4966
Test score: 0.35
-------------------------------
KNN feature eval
Train score: 0.4036
Test score: 0.300125


## Protocol #2 Logisitc Regression

In [12]:
model = ResNetSimCLR(out_dim=config['out_dim'])
model.eval()

state_dict = torch.load(os.path.join(checkpoints_folder, 'model.pth'), map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

<All keys matched successfully>

In [13]:
def next_batch(X, y, batch_size):
    for i in range(0, X.shape[0], batch_size):
        X_batch = torch.tensor(X[i: i+batch_size]) / 255.
        y_batch = torch.tensor(y[i: i+batch_size])
        yield X_batch.permute((0,3,1,2)), y_batch

In [14]:
X_train_feature = []

for batch_x, batch_y in next_batch(X_train, y_train, batch_size=128):
    features, _ = model(batch_x)
    X_train_feature.extend(features.detach().numpy())
    
X_train_feature = np.array(X_train_feature)

print("Train features")
print(X_train_feature.shape)

Train features
(5000, 512)


In [15]:
X_test_feature = []

for batch_x, batch_y in next_batch(X_test, y_test, batch_size=256):
    features, _ = model(batch_x)
    X_test_feature.extend(features.detach().numpy())
    
X_test_feature = np.array(X_test_feature)

print("Test features")
print(X_test_feature.shape)

Test features
(8000, 512)


In [16]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train_feature)

linear_model_eval(scaler.transform(X_train_feature), y_train, scaler.transform(X_test_feature), y_test)

/home/thalles/anaconda3/envs/pytorch/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Logistic Regression feature eval
Train score: 0.9628
Test score: 0.75
-------------------------------
KNN feature eval
Train score: 0.7764
Test score: 0.709125


In [17]:
# SimCLR feature evaluation
# Train score: 0.8966
# Test score: 0.634125